In [9]:
!pip install pyserial

In [54]:
import serial

START_TEST        = 0b100
BEFORE_SOLENOID   = 0b001
AFTER_SOLENOID    = 0b010
BEAM_READ         = 0b011 
END_TEST          = 0b110   


def read_data(ser):
    while True:
        if ser.in_waiting >= 7:
            data = ser.read(7)
            identifier = (data[0] >> 5) & 0x03

            if identifier == BEAM_READ:
                analog_value = ((data[0] & 0x1F) << 5) | ((data[1] >> 3) & 0x1F)
            else:
                analog_value = None
            time_micros = (data[2] << 24) | (data[3] << 16) | (data[4] << 8) | data[5]
            
            
            delimiter = data[6] & 0xFF
                
            if delimiter == 0xFF:
                print(f"Identifier: {identifier}, Analog Value: {analog_value}, Time: {time_micros}")
            else:
                print("Delimiter not found, packet discarded.")

if __name__ == "__main__":
    ser = serial.Serial('/dev/cu.usbserial-110', 9600)  # Adjust the port as needed
    read_data(ser)


Identifier: 3, Analog Value: 1023, Time: 314137336
Identifier: 3, Analog Value: 1023, Time: 314932616
Identifier: 3, Analog Value: 1023, Time: 315905112
Identifier: 3, Analog Value: 1023, Time: 316490942
